In [1]:
import onnx
from onnx import helper, numpy_helper, shape_inference
import onnxruntime as ort
from pathlib import Path


In [2]:
ONNX_IN  = Path("/home/hschatzle/monte-carlo-selection/data/models/resnet50_tl_20250829.onnx")
ONNX_OUT = Path("/home/hschatzle/monte-carlo-selection/data/models/resnet50_tl_20250829_with_feats.onnx")


model = onnx.load(str(ONNX_IN))

# logits tensor is the current graph output name (your model: "output")
assert len(model.graph.output) == 1, "Expected single output for logits"
logits_src = model.graph.output[0].name
print("Logits source tensor:", logits_src)

# penultimate features tensor: last Flatten output (right before Gemm)
flatten_nodes = [n for n in model.graph.node if n.op_type == "Flatten"]
assert flatten_nodes, "No Flatten node found"
feats_src = flatten_nodes[-1].output[0]
print("Feats source tensor:", feats_src)

Logits source tensor: output
Feats source tensor: /Flatten_output_0


In [3]:
# Cell 2. Add Identity heads for both outputs (stable naming) + set graph outputs (feats + logits)
import onnx
from onnx import helper, TensorProto, shape_inference

# Infer shapes if possible (so outputs have known shapes)
model_inf = shape_inference.infer_shapes(model)

def find_vi(name: str):
    # Search existing value_info / inputs / outputs
    for vi in list(model_inf.graph.value_info) + list(model_inf.graph.input) + list(model_inf.graph.output):
        if vi.name == name:
            return vi
    return None

def make_output_vi(src_name: str, out_name: str):
    vi = find_vi(src_name)
    if vi is None:
        # Fallback: unknown shape
        return helper.make_tensor_value_info(out_name, TensorProto.FLOAT, None)

    # Copy type/shape from src, but rename to out_name
    t = vi.type.tensor_type
    elem_type = t.elem_type
    shape = [d.dim_value if d.dim_value > 0 else (d.dim_param if d.dim_param else None) for d in t.shape.dim]
    # ONNX wants dim_value or dim_param, cannot mix None directly. If unknown, omit the whole shape.
    if any(s is None for s in shape):
        return helper.make_tensor_value_info(out_name, elem_type, None)
    return helper.make_tensor_value_info(out_name, elem_type, shape)

# Create explicit output names
FEATS_OUT  = "features"
LOGITS_OUT = "logits"

# Add Identity nodes to expose them as named outputs
id_feats  = helper.make_node("Identity", inputs=[feats_src],  outputs=[FEATS_OUT],  name="ExposeFeats")
id_logits = helper.make_node("Identity", inputs=[logits_src], outputs=[LOGITS_OUT], name="ExposeLogits")

model.graph.node.extend([id_feats, id_logits])

# Replace graph outputs with both
model.graph.ClearField("output")
model.graph.output.extend([
    make_output_vi(feats_src, FEATS_OUT),
    make_output_vi(logits_src, LOGITS_OUT),
])

onnx.save(model, str(ONNX_OUT))
print("Saved:", ONNX_OUT)


Saved: /home/hschatzle/monte-carlo-selection/data/models/resnet50_tl_20250829_with_feats.onnx


In [4]:
# Cell 3. Verify with ONNX Runtime
import onnxruntime as ort

sess = ort.InferenceSession(str(ONNX_OUT), providers=["CPUExecutionProvider"])

print("Outputs:")
for o in sess.get_outputs():
    print(" ", o.name, o.shape, o.type)

print("\nInputs:")
for i in sess.get_inputs():
    print(" ", i.name, i.shape, i.type)


Outputs:
  features ['batch_size', 2048] tensor(float)
  logits ['batch_size', 54] tensor(float)

Inputs:
  input ['batch_size', 3, 224, 224] tensor(float)
